In [4]:
import math
from datetime import datetime
import pytz  # For handling timezones

def calculate_solar_position(latitude, longitude, date_time):
    """
    Calculate the solar elevation and azimuth angles.

    Args:
        latitude (float): Latitude of the location (in degrees).
        longitude (float): Longitude of the location (in degrees).
        date_time (datetime): Local date and time (timezone-aware).

    Returns:
        dict: Solar elevation and azimuth angles (in degrees).
    """
    # Constants
    PI = math.pi
    DEG_TO_RAD = PI / 180
    RAD_TO_DEG = 180 / PI

    # Convert date_time to UTC
    utc_time = date_time.astimezone(pytz.utc)
    
    # Day of the year (n)
    day_of_year = utc_time.timetuple().tm_yday
    
    # Declination angle (δ)
    declination = 23.45 * math.sin(DEG_TO_RAD * (360 / 365 * (day_of_year - 81)))
    
    # Time correction factor
    B = 360 / 365 * (day_of_year - 81) * DEG_TO_RAD
    E = 9.87 * math.sin(2 * B) - 7.53 * math.cos(B) - 1.5 * math.sin(B)
    time_correction = 4 * longitude + E

    # Local solar time (LST)
    solar_time = date_time.hour * 60 + date_time.minute + date_time.second / 60 + time_correction
    hour_angle = (solar_time / 60 - 12) * 15  # Hour angle (H) in degrees
    
    # Latitude to radians
    latitude_rad = latitude * DEG_TO_RAD
    declination_rad = declination * DEG_TO_RAD
    hour_angle_rad = hour_angle * DEG_TO_RAD

    # Solar zenith angle (θz)
    cos_theta_z = (
        math.sin(latitude_rad) * math.sin(declination_rad) +
        math.cos(latitude_rad) * math.cos(declination_rad) * math.cos(hour_angle_rad)
    )
    theta_z = math.acos(cos_theta_z) * RAD_TO_DEG
    elevation_angle = 90 - theta_z

    # Solar azimuth angle (γs)
    sin_gamma_s = math.cos(declination_rad) * math.sin(hour_angle_rad) / math.cos(theta_z * DEG_TO_RAD)
    cos_gamma_s = (
        math.sin(elevation_angle * DEG_TO_RAD) * math.sin(latitude_rad) - math.sin(declination_rad)
    ) / (math.cos(elevation_angle * DEG_TO_RAD) * math.cos(latitude_rad))
    azimuth_angle = math.atan2(sin_gamma_s, cos_gamma_s) * RAD_TO_DEG

    # Adjust azimuth angle to [0, 360] range
    if azimuth_angle < 0:
        azimuth_angle += 360

    return {
        "elevation_angle": elevation_angle,
        "azimuth_angle": azimuth_angle
    }

# Interactive Inputs
def main():
    print("Enter your location details:")
    latitude = float(input("Latitude (in degrees): "))
    longitude = float(input("Longitude (in degrees): "))
    timezone_str = input("Timezone (e.g., 'Asia/Kolkata'): ")

    print("\nEnter the date and time for the calculation:")
    year = int(input("Year (e.g., 2024): "))
    month = int(input("Month (e.g., 3 for March): "))
    day = int(input("Day (e.g., 21): "))
    hour = int(input("Hour (24-hour format, e.g., 9): "))
    minute = int(input("Minute (e.g., 30): "))

    # Create a timezone-aware datetime object
    local_timezone = pytz.timezone(timezone_str)
    date_time = local_timezone.localize(datetime(year, month, day, hour, minute))

    # Calculate solar position
    solar_position = calculate_solar_position(latitude, longitude, date_time)

    # Output results
    print("\nSolar Position Results:")
    print(f"Solar Elevation Angle: {solar_position['elevation_angle']:.2f}°")
    print(f"Solar Azimuth Angle: {solar_position['azimuth_angle']:.2f}°")

if __name__ == "__main__":
    main()

    
"""
Elevation Angle = How high the sun is.
Vertical measurement (upwards from the horizon).
Azimuth Angle = Which direction to face to see the sun.
Horizontal measurement (compass direction).
Example to Compare:

At sunrise:
The elevation angle is small, maybe 2°, because the sun is just above the horizon.
The azimuth angle is around 90°, meaning the sun is in the East.
At noon:
The elevation angle is high, like 70°, because the sun is almost overhead.
The azimuth angle is close to 180°, meaning the sun is in the South (Northern Hemisphere).

""""

Enter your location details:
Latitude (in degrees): 17.57
Longitude (in degrees): 79.69
Timezone (e.g., 'Asia/Kolkata'): Asia/Kolkata

Enter the date and time for the calculation:
Year (e.g., 2024): 2024
Month (e.g., 3 for March): 11
Day (e.g., 21): 25
Hour (24-hour format, e.g., 9): 11
Minute (e.g., 30): 10

Solar Position Results:
Solar Elevation Angle: 11.03°
Solar Azimuth Angle: 84.38°
